In [1]:
%run utils/devtools.py

In [2]:
%reloadmypkg utils

import pandas as pd
from utils.url2platform import *
from utils.standardise_url import *
from utils.fetch_info import *
from utils.dump_df_to_json import *

✅ Reloaded package 'utils' and its submodules.


In [3]:
sheet_path = "../raw-data/striver/sde-sheet-response-data.json"
export_file = "../cleaned-data/striver/sde.json"

In [4]:
with open(sheet_path) as file:
    raw = json.load(file)

flattened_data = [
    {
        "gfg": topic["gfg_link"],
        "c360": topic["cs_link"],
        "lc": topic["lc_link"],
        "original_title": topic["title"],
        "yt": topic["yt_link"],
        "post": topic["post_link"],
        "step_title": topic["head_step_no"]
    }
    for step in raw["sheetData"]
    for topic in step["topics"]
]

df = pd.DataFrame(flattened_data)

In [5]:
def get_best_url(row):
    return (
        row["lc"] or
        row["gfg"] or
        row["c360"] or
        row["yt"] or
        row["post"]
    )

In [6]:
df["url"] = df.apply(get_best_url, axis=1)

In [7]:
df = df[["url", "original_title", "step_title"]]

In [8]:
df.shape

(191, 3)

In [9]:
missing_urls = df[df["url"].isna()]
missing_urls.shape

(0, 3)

In [10]:
df.loc[90]

url               https://leetcode.com/accounts/login/?next=/pro...
original_title                                The Celebrity Problem
step_title                                  Stack and Queue Part-II
Name: 90, dtype: object

In [11]:
# Replace premium LeetCode problem with LintCode link
df.at[90, "url"] = "https://www.lintcode.com/problem/645/"

In [12]:
redirect_urls = df[df["url"].str.contains("implement-strstr")]
redirect_urls.shape

(2, 3)

In [13]:
# Fix the redirect urls
df.at[97, "url"] = "https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/"
df.at[98, "url"] = "https://leetcode.com/problems/find-the-index-of-the-first-occurrence-in-a-string/"

In [14]:
# Find the duplicates

dupes_df = df[df.duplicated(subset="url", keep=False)]
dupes_df.shape

(20, 3)

In [15]:
dupe_counts = dupes_df.groupby("url").size()
print(f"Total unique URLs with duplicates: {len(dupe_counts)}")

Total unique URLs with duplicates: 9


In [16]:
import nest_asyncio
nest_asyncio.apply()

standardised_df = await standardise_urls_async(df, "striver-sde-urls-expanded.json")

Filter list is empty. Trying to expand all 191 URLs.

Processing batch 1/1
Loaded 0 results from checkpoint
Processing 191 pending URLs in this batch


Expanding URLs: 100%|██████████| 191/191 [00:02<00:00, 70.42it/s] 

Progress saved to striver-sde-urls-expanded.json
----------------------------------------------------------------------
Progress saved to striver-sde-urls-expanded.json
----------------------------------------------------------------------

191/191 URLs expanded in 5.72 seconds.


In [17]:
standardised_df.shape

(191, 5)

In [18]:
# Adding type column
standardised_df['type'] = standardised_df['expanded_stripped_url'].apply(
    lambda url: "problem" if is_problem_url(url) else "tutorial"
)

In [19]:
standardised_df.shape

(191, 6)

In [20]:
standardised_df.head(1)

,url,original_title,step_title,expanded_url,expanded_stripped_url,type
0,https://leetcode.com/problems/set-matrix-zeroes/,Set Matrix Zeros,Arrays,https://leetcode.com/problems/set-matrix-zeroes/,https://leetcode.com/problems/set-matrix-zeroes/,problem


In [21]:
# Adding id_base, id, title, platform columns
import nest_asyncio
nest_asyncio.apply()

final_df = await fetch_info(standardised_df, "striver-sde-info-added.json")

Found 191 URLs.

Batch 1/1: items 0-190
Loaded 0 from checkpoint
Processing 191 URLs...


Fetching items: 100%|██████████| 191/191 [00:03<00:00, 63.27it/s] 

Saved progress (191 items) to striver-sde-info-added.json
------------------------------
Saved progress (191 items) to striver-sde-info-added.json
------------------------------

Fetched 191/191 items in 3.1s.


In [22]:
final_df.shape

(191, 10)

In [23]:
dump_df_to_json(final_df, export_file)